In [1]:
%pip install h2o

  Using cached h2o-3.38.0.3.tar.gz (177.5 MB)
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=347c6bfd5eead1d468c03e00dfb962870164f761641bc87413ea1865f3e38171
  Stored in directory: c:\users\alvar\appdata\local\pip\cache\wheels\42\8a\04\a258412102066310d1efaf9d7db94c3b3e6040a85da5bb3886
Successfully built h2o
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import pylab as plt
import seaborn as sns

%matplotlib inline   

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

from sklearn.linear_model import LogisticRegression            # clasificar, logistic regression model   

from sklearn.preprocessing import StandardScaler               # standarized             

from sklearn.model_selection import train_test_split as tts  

from sklearn.metrics import mean_squared_error as mse 
from sklearn.linear_model import LinearRegression as LinReg

from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet   # regularizacion L1+L2
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR

from xgboost import XGBRegressor as XGBR

from catboost import CatBoostRegressor as CTR

from lightgbm import LGBMRegressor as LGBMR
from sklearn.cluster import DBSCAN
import h2o

from h2o.automl import H2OAutoML

In [3]:
df=pd.read_csv('../data/salaries_data.csv')
test=pd.read_csv('../data/testeo.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [4]:
residence = []
for i in df.employee_residence:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        residence.append('OTHER_COUNTRY')
    else:
        residence.append(i)
df.employee_residence=residence

In [5]:
df.employment_type.value_counts()

FT    484
PT      8
CT      4
FL      4
Name: employment_type, dtype: int64

In [6]:
df.employment_type=LabelEncoder().fit_transform(df.employment_type)

In [7]:
df.employee_residence=LabelEncoder().fit_transform(df.employee_residence)

In [8]:
df=pd.get_dummies(df, columns=['work_year'], drop_first=True)
df.head()

,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,work_year_2021,work_year_2022
0,SE,2,Data Engineer,140250,USD,140250,8,100,US,M,0,1
1,SE,2,Data Engineer,135000,USD,135000,8,100,US,M,0,1
2,MI,2,BI Data Analyst,100000,USD,100000,8,100,US,M,1,0
3,MI,0,ML Engineer,270000,USD,270000,8,100,US,L,1,0
4,MI,2,Data Engineer,22000,EUR,26005,7,0,US,L,1,0


In [9]:
df=pd.get_dummies(df, columns=['experience_level'], drop_first=True)
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE
0,2,Data Engineer,140250,USD,140250,8,100,US,M,0,1,0,0,1
1,2,Data Engineer,135000,USD,135000,8,100,US,M,0,1,0,0,1
2,2,BI Data Analyst,100000,USD,100000,8,100,US,M,1,0,0,1,0
3,0,ML Engineer,270000,USD,270000,8,100,US,L,1,0,0,1,0
4,2,Data Engineer,22000,EUR,26005,7,0,US,L,1,0,0,1,0


In [10]:
df=pd.get_dummies(df, columns=['remote_ratio'])
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,company_size,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,remote_ratio_0,remote_ratio_50,remote_ratio_100
0,2,Data Engineer,140250,USD,140250,8,US,M,0,1,0,0,1,0,0,1
1,2,Data Engineer,135000,USD,135000,8,US,M,0,1,0,0,1,0,0,1
2,2,BI Data Analyst,100000,USD,100000,8,US,M,1,0,0,1,0,0,0,1
3,0,ML Engineer,270000,USD,270000,8,US,L,1,0,0,1,0,0,0,1
4,2,Data Engineer,22000,EUR,26005,7,US,L,1,0,0,1,0,1,0,0


In [11]:
countries = []
for i in df.company_location:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        countries.append('OTHER_COUNTRY')
    else:
        countries.append(i)
df.company_location=countries

In [12]:
df.company_location.value_counts()

US               296
OTHER_COUNTRY     62
GB                41
CA                25
DE                23
IN                19
FR                12
ES                12
GR                10
Name: company_location, dtype: int64

In [13]:
df.job_title=LabelEncoder().fit_transform(df.job_title)

In [14]:
df.salary_currency.value_counts()

USD    329
EUR     78
GBP     38
INR     23
CAD     16
JPY      3
TRY      3
AUD      2
PLN      2
MXN      1
CHF      1
CLP      1
BRL      1
HUF      1
SGD      1
Name: salary_currency, dtype: int64

In [15]:
df.salary_currency=LabelEncoder().fit_transform(df. salary_currency)

In [16]:
df.salary_currency.value_counts()

14    329
5      78
6      38
8      23
2      16
9       3
13      3
0       2
11      2
10      1
3       1
4       1
1       1
7       1
12      1
Name: salary_currency, dtype: int64

In [17]:
df.company_location=LabelEncoder().fit_transform(df.company_location)

In [18]:
df=pd.get_dummies(df, columns=['company_size'], drop_first=True)
df.head()

,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,remote_ratio_0,remote_ratio_50,remote_ratio_100,company_size_M,company_size_S
0,2,17,140250,14,140250,8,8,0,1,0,0,1,0,0,1,1,0
1,2,17,135000,14,135000,8,8,0,1,0,0,1,0,0,1,1,0
2,2,5,100000,14,100000,8,8,1,0,0,1,0,0,0,1,1,0
3,0,36,270000,14,270000,8,8,1,0,0,1,0,0,0,1,0,0
4,2,17,22000,5,26005,7,8,1,0,0,1,0,1,0,0,0,0


In [19]:
df.columns

Index(['employment_type', 'job_title', 'salary', 'salary_currency',
       'salary_in_usd', 'employee_residence', 'company_location',
       'work_year_2021', 'work_year_2022', 'experience_level_EX',
       'experience_level_MI', 'experience_level_SE', 'remote_ratio_0',
       'remote_ratio_50', 'remote_ratio_100', 'company_size_M',
       'company_size_S'],
      dtype='object')

In [20]:
test.drop('salary', axis=1, inplace=True)

In [21]:
test=pd.get_dummies(test, columns=['experience_level'], drop_first=True)

test.employment_type=LabelEncoder().fit_transform(test.employment_type)

test.job_title=LabelEncoder().fit_transform(test.job_title)

test.salary_currency=LabelEncoder().fit_transform(test.salary_currency)

residence = []
for i in test.employee_residence:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        residence.append('OTHER_COUNTRY')
    else:
        residence.append(i)
test.employee_residence=residence

test.employee_residence=LabelEncoder().fit_transform(test.employee_residence)

countries = []
for i in test.company_location:
    if i not in ['US', 'GB', 'CA', 'DE', 'IN', 'FR', 'ES', 'GR']:
        countries.append('OTHER_COUNTRY')
    else:
        countries.append(i)
test.company_location=countries

test.company_location=LabelEncoder().fit_transform(test.company_location)

test=pd.get_dummies(test, columns=['company_size'], drop_first=True)

test=pd.get_dummies(test, columns=['work_year'], drop_first=True)

test=pd.get_dummies(test, columns=['remote_ratio'])
test.head()

,employment_type,job_title,salary_currency,employee_residence,company_location,experience_level_EX,experience_level_MI,experience_level_SE,company_size_M,company_size_S,work_year_2021,work_year_2022,remote_ratio_0,remote_ratio_50,remote_ratio_100
0,1,22,11,7,7,0,0,1,0,1,0,0,1,0,0
1,1,18,11,8,8,0,1,0,0,0,0,0,0,0,1
2,1,8,11,8,8,0,1,0,0,0,0,0,0,0,1
3,1,20,2,7,7,0,1,0,1,0,0,0,1,0,0
4,1,27,7,6,6,0,1,0,0,0,0,0,0,0,1


In [22]:
test.shape, df.shape

((107, 15), (500, 17))

In [23]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.51-b03, mixed mode)
  Starting server from C:\Users\alvar\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\alvar\AppData\Local\Temp\tmpnwsh43px
  JVM stdout: C:\Users\alvar\AppData\Local\Temp\tmpnwsh43px\h2o_alvar_started_from_python.out
  JVM stderr: C:\Users\alvar\AppData\Local\Temp\tmpnwsh43px\h2o_alvar_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_alvar_wppsg8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.514 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [25]:
h2train=h2o.H2OFrame(df)
h2test=h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [26]:
h2train.columns

['employment_type',
 'job_title',
 'salary',
 'salary_currency',
 'salary_in_usd',
 'employee_residence',
 'company_location',
 'work_year_2021',
 'work_year_2022',
 'experience_level_EX',
 'experience_level_MI',
 'experience_level_SE',
 'remote_ratio_0',
 'remote_ratio_50',
 'remote_ratio_100',
 'company_size_M',
 'company_size_S']

In [27]:
X=[c for c in h2train.columns if c!='salary' and c!='salary_in_usd']

y='salary_in_usd'

In [28]:
automl=H2OAutoML(max_models=50,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [29]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |
15:33:43.377: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_1_20221205_153343


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    36                 36                          13168                  6            10           8.11111       19            29            24.3611

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1491864791.5697777
RMSE: 38624.66558521611
MAE: 26339.854801757814
RMSLE: 0.40226886066068124
Mean Residual Deviance: 1491864791.5697777

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2321791374.71011
RMSE: 48184.9704234641
MAE: 32547.234180032356
RMSLE: 0.5122277172362714
Mean Residual Deviance: 2321791374.71011

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     32747.1      2819.62      36662.7       32071.3       28779.3       33064.4       33157.8
mean_residual_deviance  2.35063e+09  6.78335e+08  3.2502e+09    1.89981e+09   1.51245e+09   2.38769e+09   2.70301e+09
mse                     2.35063e+09  6.78335e+08  3.2502e+09    1.89981e+09   1.51245e+09   2.38769e+09   2.70301e+09
r2                      0.514084     0.0523186    0.432993      0.548413      0.531568      0.563797      0.493647
residual_deviance       2.35063e+09  6.78335e+08  3.2502e+09    1.89981e+09   1.51245e+09   2.38769e+09   2.70301e+09
rmse                    48068.4      7076.38      57010.5       43586.8       38890.2       48864         51990.5
rmsle                   0.507187     0.0563173    0.435533      0.587864      0.524921      0.480189      0.507427

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2022-12-05 15:33:45  0.186 sec   0                  69544.1          52000.6         4.83638e+09
    2022-12-05 15:33:45  0.195 sec   5                  55894.2          39986.2         3.12416e+09
    2022-12-05 15:33:45  0.203 sec   10                 48871.8          33918           2.38846e+09
    2022-12-05 15:33:45  0.211 sec   15                 45267.7          30869.5         2.04916e+09
    2022-12-05 15:33:45  0.220 sec   20                 42856.2          28889.8         1.83665e+09
    2022-12-05 15:33:45  0.229 sec   25                 41152.9          27735.7         1.69356e+09
    2022-12-05 15:33:45  0.237 sec   30                 39782.5          27004.3         1.58265e+09
    2022-12-05 15:33:45  0.245 sec   35                 38847.1          26450.2         1.50909e+09
    2022-12-05 15:33:45  0.247 sec   36                 38624.7          26339.9         1.49186e+09

Variable Importances: 
variable             relative_importance    scaled_importance    percentage
-------------------  ---------------------  -------------------  ------------
employee_residence   2.79744e+12            1                    0.325232
job_title            1.81541e+12            0.648956             0.211061
company_location     1.762e+12              0.629863             0.204852
experience_level_SE  4.39228e+11            0.157011             0.051065
experience_level_EX  3.62125e+11            0.129449             0.0421008
company_size_S       2.45637e+11            0.0878079            0.0285579
salary_currency      2.3793e+11             0.085053             0.0276619
work_year_2022       2.15086e+11            0.0768869      

In [30]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_1_20221205_153343,48185,2.32179e+09,32547.2,0.512228,2.32179e+09
GBM_grid_1_AutoML_1_20221205_153343_model_2,48291.1,2.33203e+09,32438.4,0.509382,2.33203e+09
GBM_grid_1_AutoML_1_20221205_153343_model_15,48476.3,2.34995e+09,32593.3,0.504458,2.34995e+09
GBM_grid_1_AutoML_1_20221205_153343_model_11,48531.5,2.35531e+09,32702.4,0.501994,2.35531e+09
GBM_2_AutoML_1_20221205_153343,48631,2.36498e+09,32900.9,0.5123,2.36498e+09
GBM_grid_1_AutoML_1_20221205_153343_model_13,48745.2,2.37609e+09,32389.4,0.50643,2.37609e+09
GBM_3_AutoML_1_20221205_153343,48762.6,2.37779e+09,32628.7,0.512422,2.37779e+09
GBM_grid_1_AutoML_1_20221205_153343_model_10,48993.1,2.40033e+09,33173.8,0.512551,2.40033e+09
GBM_grid_1_AutoML_1_20221205_153343_model_4,49196.9,2.42033e+09,32829.8,0.519725,2.42033e+09
GBM_grid_1_AutoML_1_20221205_153343_model_9,49248.7,2.42543e+09,33088.1,0.518121,2.42543e+09


In [31]:
y_pred=automl.leader.predict(h2test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [32]:
y_pred[:10]

predict
81400.8
120504
132464
48503.7
58694.5
61983.1
126335
65343.7
37743
155592


In [33]:
res=pd.read_csv('../data/muestra.csv')

In [34]:
res.head()

,id,salary_in_usd
0,0,70377
1,1,85839
2,2,63613
3,3,88561
4,4,37621


In [40]:
df_final = y_pred.as_data_frame()

In [42]:
res.salary_in_usd=df_final.predict

In [43]:
res.head()

,id,salary_in_usd
0,0,81400.847698
1,1,120504.188215
2,2,132463.589604
3,3,48503.732945
4,4,58694.530545


In [44]:
res.to_csv('resultado1.csv',index=False)